In [1]:
import numpy as np
from bnt_smooth import LognormalWeakLensingSim  # your class file

# --- Define a parent n(z) and tomographic binning ---
def parent_nz(z):
    return z**2 * np.exp(-(z / 0.5)**1.5)

def make_equal_ngal_bins(nz_func, z, nbins):
    from scipy.interpolate import interp1d
    nz = nz_func(z)
    nz /= np.trapezoid(nz, z)
    cdf = np.cumsum(nz)
    cdf /= cdf[-1]
    inv_cdf = interp1d(np.concatenate([[0], cdf, [1]]),
                       np.concatenate([[z[0]], z, [z[-1]]]))
    edges = inv_cdf(np.linspace(0, 1, nbins + 1))
    bins = []
    for i in range(nbins):
        mask = (z >= edges[i]) & (z <= edges[i+1])
        bins.append((z[mask], nz[mask]))
    return bins, edges

# --- Simulation settings ---
z = np.linspace(0.01, 2.5, 500)
nz_list, _ = make_equal_ngal_bins(parent_nz, z, nbins=3)
n_eff_list = [30.0] * 3
sigma_eps_list = [0.26] * 3
baryon_feedback = 3.13
sigma8 = 0.8
seed = 1234
l_max = 256
lognormal_shift = 1.0
zmax = 3.0
nslices = 50

# --- Initialize simulation ---
sim = LognormalWeakLensingSim(
    nz_list=nz_list,
    n_eff_list=n_eff_list,
    sigma_eps_list=sigma_eps_list,
    baryon_feedback=baryon_feedback,
    sigma8=sigma8,
    lognormal_shift=lognormal_shift,
    seed=seed,
    l_max=l_max,
    zmax=zmax,
    nslices=nslicesimport healpy as hp
import matplotlib.pyplot as plt

# --- Plot the first map (shell 0) ---
hp.mollview(maps[0], title="Lognormal Matter Map (Shell 0)", unit="δ", norm='hist')
plt.show()
)

# --- Generate lognormal matter fields from scratch ---
maps = sim.generate_matter_fields_from_scratch()

# --- Print summary ---
print(f"Generated {len(maps)} lognormal HEALPix maps.")
print(f"Each map has {maps[0].size} pixels (nside = {sim.nside})")

/Users/taylor.4264/test/BNTSmooth/bnt_smooth.py:126: HealpyDeprecationWarning: "verbose" was deprecated in version 1.15.0 and will be removed in a future version. 
  delta_g = hp.synfast(full_cl, nside=nside, verbose=False)


Generated 50 lognormal HEALPix maps.
Each map has 786432 pixels (nside = 256)


In [ ]:
import healpy as hp
import matplotlib.pyplot as plt

# --- Plot the first map (shell 0) ---
hp.mollview(maps[4], title="Lognormal Matter Map (Shell 0)", unit="δ", norm='hist')
plt.show()


In [14]:
cosmo

NameError: name 'cosmo' is not defined

In [2]:
ell, cl = sim.compute_wl_cls()

In [3]:
np.shape(cl)

(6, 3001)

In [4]:
ell, cls = sim.compute_wl_cls()
cl_ell5 = get_cl_matrix_at_ell(ell, cls, ell_target=5, nbins=sim.nbins)
print(cl_ell5)

NameError: name 'get_cl_matrix_at_ell' is not defined

In [5]:
def is_positive_definite(matrix):
    """
    Check whether a matrix is positive-definite.

    Parameters
    ----------
    matrix : ndarray
        Square symmetric matrix to check.

    Returns
    -------
    is_pd : bool
        True if matrix is positive-definite, False otherwise.
    """
    try:
        np.linalg.cholesky(matrix)
        return True
    except np.linalg.LinAlgError:
        return False

In [8]:
if is_positive_definite(cl_ell5):
    print("C_ell=5 is positive-definite ✅")
else:
    print("C_ell=5 is NOT positive-definite ❌")

C_ell=5 is positive-definite ✅


In [1]:

def check_all_cls_positive_definite(ell_array, cls_list, nbins):
    """
    Check whether each Cl^{ij} matrix at each ell is positive-definite.

    Parameters
    ----------
    ell_array : ndarray
        Array of ell values.
    cls_list : list of ndarray
        Flattened lower-triangular list of Cl arrays.
    nbins : int
        Number of tomographic bins.
    """
    n_ell = len(ell_array)
    for i_ell in range(n_ell):
        cl_matrix = np.zeros((nbins, nbins))
        idx = 0
        for i in range(nbins):
            for j in range(i + 1):
                cl_matrix[i, j] = cls_list[idx][i_ell]
                cl_matrix[j, i] = cl_matrix[i, j]
                idx += 1

        try:
            np.linalg.cholesky(cl_matrix)
            print(f"ell = {ell_array[i_ell]:4d}: ✅ positive-definite")
        except np.linalg.LinAlgError:
            print(f"ell = {ell_array[i_ell]:4d}: ❌ NOT positive-definite")


'\ndef check_all_cls_positive_definite(ell_array, cls_list, nbins):\n    """\n    Check whether each Cl^{ij} matrix at each ell is positive-definite.\n\n    Parameters\n    ----------\n    ell_array : ndarray\n        Array of ell values.\n    cls_list : list of ndarray\n        Flattened lower-triangular list of Cl arrays.\n    nbins : int\n        Number of tomographic bins.\n    """\n    n_ell = len(ell_array)\n    for i_ell in range(n_ell):\n        cl_matrix = np.zeros((nbins, nbins))\n        idx = 0\n        for i in range(nbins):\n            for j in range(i + 1):\n                cl_matrix[i, j] = cls_list[idx][i_ell]\n                cl_matrix[j, i] = cl_matrix[i, j]\n                idx += 1\n\n        try:\n            np.linalg.cholesky(cl_matrix)\n            print(f"ell = {ell_array[i_ell]:4d}: ✅ positive-definite")\n        except np.linalg.LinAlgError:\n            print(f"ell = {ell_array[i_ell]:4d}: ❌ NOT positive-definite")\n'

In [2]:
'''
ell, cls = sim.compute_wl_cls()
check_all_cls_positive_definite(ell, cls, nbins=sim.nbins)
'''

'\nell, cls = sim.compute_wl_cls()\ncheck_all_cls_positive_definite(ell, cls, nbins=sim.nbins)\n'

In [15]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

# use the CAMB cosmology that generated the matter power spectra
import camb
from cosmology import Cosmology

In [16]:


import healpy as hp
import matplotlib.pyplot as plt
import numpy as np

# use the CAMB cosmology that generated the matter power spectra
import camb
from cosmology import Cosmology

# almost all GLASS functionality is available from the `glass` namespace
import glass
import glass.ext.camb

# how many arcmin2 over the entire sphere
ARCMIN2_SPHERE = 60**6 // 100 / np.pi

# creating a numpy random number generator for sampling
rng = np.random.default_rng(seed=42)

# cosmology for the simulation
h = 0.7
Oc = 0.25
Ob = 0.05

# basic parameters of the simulation
nside = lmax = 256

# set up CAMB parameters for matter angular power spectrum
pars = camb.set_params(
    H0=100 * h,
    omch2=Oc * h**2,
    ombh2=Ob * h**2,
    NonLinear=camb.model.NonLinear_both,
)

# get the cosmology from CAMB
cosmo = Cosmology.from_camb(pars)

# shells of 200 Mpc in comoving distance spacing
zb = glass.distance_grid(cosmo, 0.0, 1.0, dx=200.0)

# linear window function for shells
shells = glass.linear_windows(zb)

# compute the angular matter power spectra of the shells with CAMB
cls = glass.ext.camb.matter_cls(pars, lmax, shells)



     


In [17]:
np.shape(cls)

(136, 257)

In [20]:
cosmo